In [29]:
from dotenv import load_dotenv
from openai import OpenAI

import pandas as pd
import numpy as np
import pickle
import faiss
import os

load_dotenv() 

EMBEDER_KEY = os.getenv('EMBEDER_KEY')

# 0. Подготовка

In [3]:
with open('data/prepared_data_frame.pickle', 'rb') as f:
    data = pickle.load(f)

In [7]:
def data_to_storage(id_series: pd.Series, data_series: pd.Series):
    '''Функция парсит данные из датасета, собирая в JSON БД
    
    Args:
        id_series - столбец датасета с идентификатором документа
        data_series - стобец датасета с чанками
    Output:
        JSON БД
    '''
    storage = {}
    key_id = 0

    for row_num in range(len(data_series)):
        id_doc = id_series[row_num]
        data_document = data_series[row_num]

        for document in data_document:
            # Проверка на дополнительную вложенность
            if isinstance(document, list):
                for chunk in document:

                    storage[key_id] = (id_doc, chunk)
                    key_id += 1
            else:
                storage[key_id] = (id_doc, document)
                key_id += 1

    print(f"Storage ready, key from 0 to {key_id-1}")

    return storage

In [34]:
def get_embedding(text):
    # Подключаемся к модели
    client = OpenAI(
        # Базовый url - сохранять без изменения
        base_url="https://ai-for-finance-hack.up.railway.app/",
        # Указываем наш ключ, полученный ранее
        api_key=EMBEDER_KEY,
    )
    # Формируем запрос к клиенту
    response = client.embeddings.create(
        # Выбираем любую допступную модель из предоставленного списка
        model="text-embedding-3-small",
        # Отправяем запрос
        input=text, 
        # Определяем размерность эмбединга
        dimensions = 512
    )
    # Формируем ответ на запрос и возвращаем его в результате работы функции
    return response.data[0].embedding


In [13]:
def get_batch_embeddings(texts, batch_size=32, dimensions=512):
    """Батчевые запросы к embedding API

    Args:
        texts: список строк
        batch_size: количество текстов в одном запросе
        dimensions: размерность эмбединга
    Returns:
        Список эмбеддингов (list[list[float]]).
    """
    client = OpenAI(
        base_url="https://ai-for-finance-hack.up.railway.app/",
        api_key=EMBEDER_KEY,
    )

    embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]

        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=batch,            
            dimensions=dimensions
        )

        # каждая запись в response.data соответствует одному элементу из batch
        batch_embeddings = [item.embedding for item in response.data]
        embeddings.extend(batch_embeddings)

    return embeddings

In [30]:
def normalize_vector(embedding: np.ndarray) -> np.ndarray:
    """
    Normalizes a given vector to have unit length.

    Args:
        embedding (np.ndarray): A NumPy array representing the vector to normalize.

    Returns:
        np.ndarray: A normalized vector with unit length.
    """

    norm = np.linalg.norm(embedding)
    if abs(norm) >= 1e-9: #защита от взрыва и погрешности
      embedding /= norm

    return embedding

Faiss

In [31]:
# Function to build an HNSW index
def build_faiss_hnsw_index(dimension, ef_construction=200, M=32):
    """
    build_faiss_hnsw_index: Add a description here.

    Args:
        # List the arguments with types and descriptions.

    Returns:
        # Specify what the function returns.
    """
    """
    Builds a FAISS HNSW index for cosine similarity.

    Parameters:
        dimension (int): Dimensionality of the embeddings.
        ef_construction (int): Trade-off parameter between index construction speed and accuracy.
        M (int): Number of neighbors in the graph.

    Returns:
        index (faiss.IndexHNSWFlat): Initialized FAISS HNSW index.
    """
    index = faiss.IndexHNSWFlat(dimension, M)  # HNSW index
    index.hnsw.efConstruction = ef_construction  # Construction accuracy
    index.metric_type = faiss.METRIC_INNER_PRODUCT  # Cosine similarity via normalized vectors
    return index

In [32]:
# Function to populate the FAISS index
def populate_faiss_index(index: faiss.Index, documents: dict, batch_size: int=20):
    """
    populate_faiss_index: Add a description here.

    Args:
        # List the arguments with types and descriptions.

    Returns:
        # Specify what the function returns.
    """
    """
    Populates the FAISS HNSW index with normalized embeddings from the dataset.

    Parameters:
        index (faiss.Index): FAISS index to populate.
        documents (pd.Series): documents like List[list[str]]
        batch_size (int): Number of questions to process at a time.
    """
    buffer = []
    i = 0

    for _, embedding in documents.items():
        embedding = normalize_vector(embedding)
        buffer.append(embedding)
        i += 1

        # Add embeddings to the index in batches
        if len(buffer) >= batch_size:
            index.add(np.array(buffer, dtype=np.float32))
            buffer = []

    # Add remaining embeddings
    if buffer:
        index.add(np.array(buffer, dtype=np.float32))

In [41]:
# Function to perform a search query
def search_faiss_index(embeddings_storage, query, k=5):
    """
    search_faiss_index: Add a description here.

    Args:
        # List the arguments with types and descriptions.

    Returns:
        # Specify what the function returns.
    """
    """
    Searches the FAISS index for the closest matches to a query.

    Parameters:
        embeddings_storage (faiss.Index): FAISS index to search.
        query (str): Query string to search.
        k (int): Number of closest matches to retrieve.

    Returns:
        indices (np.ndarray): Indices of the top-k results.
        distances (np.ndarray): Distances of the top-k results.
    """
    # Preprocess and normalize the query embedding
    query_embedding = get_embedding(query)
    query_embedding = np.array(query_embedding, dtype=np.float32)
    query_embedding = normalize_vector(query_embedding)
    # Search the embeddings_storage
    top_k_distances, top_k_indices = embeddings_storage.search(np.array([query_embedding], dtype=np.float32), k)

    # Match return format with that used in numpy storage search
    # Note that list manipulations will give an overhead
    top_k_indices_list = top_k_indices[0].tolist()
    top_k_distances_list = top_k_distances[0].tolist()

    return top_k_indices_list, top_k_distances_list

# 1. Подготовка json БД

In [8]:
data.head()

,id,annotation,tags,text,annotation_tags_chunk,text_chunk
0,doc_001,Светлана из Казани дает частные уроки английск...,"Начать бизнес, Самозанятые, Свое дело, Налоги","[(Кто такой самозанятый?, По закону самозаняты...","[Начать бизнес, Самозанятые, Свое дело, Налоги...",[[Кто такой самозанятый?По закону самозанятый ...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","Защитить права, Банки, Банковская карта, Риски...","[(, Первым делом нужно попросить банк проверит...","[Защитить права, Банки, Банковская карта, Риск...",[[Первым делом нужно попросить банк проверить ...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"Кредиты, Долги, Просрочки, Ипотека, Кредитная ...","[(, Не переоценивайте свои финансовые возможно...","[Кредиты, Долги, Просрочки, Ипотека, Кредитная...",[[Не переоценивайте свои финансовые возможност...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","Инвестиции, Ценные бумаги, Фондовая биржа, Акц...","[(, Просто прийти на биржу и купить ценные бум...","[Инвестиции, Ценные бумаги, Фондовая биржа, Ак...",[[Просто прийти на биржу и купить ценные бумаг...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"Займы, Долги, Риски, Защитить права","[(МФО больше нет в госреестре. Значит, она зак...","[Займы, Долги, Риски, Защитить права. Вы взяли...","[[МФО больше нет в госреестре. Значит, она зак..."


In [10]:
storage_an_t = data_to_storage(
        id_series = data['id'],
        data_series = data['annotation_tags_chunk']
    )

Storage ready, key from 0 to 369


# 2. Получаем эмбединги

In [14]:
# Соберем текст для батчевого запроса
texts = list()
for _, val in storage_an_t.items():
    texts.append(val[1])

len(texts)

370

In [ ]:
# Запрос к openrouter
embeddings = get_batch_embeddings(texts)

In [ ]:
# Соберем временное хранилище векторов
embed_storage_an_t = dict()

for i in range(len(embeddings)):
    embed_storage_an_t[i] = np.array(embeddings[i], np.float32)

In [28]:
# Размерность полученных эмбедингов
embed_storage_an_t[0].shape, len(embed_storage_an_t)

((512,), 370)

# 3. Инициализируем Faiss

In [42]:
# Define the dimensions of the embedding vectors
embedding_dimension = 512  # Depends on the FastText model

# Build the HNSW index
hnsw_index_an_t = build_faiss_hnsw_index(embedding_dimension)

# Populate the index from pd.Series
populate_faiss_index(index=hnsw_index_an_t, documents=embed_storage_an_t)

In [50]:
data.loc[2]

id                                                                 doc_003
annotation               Самый надежный способ не оказаться в долгах — ...
tags                     Кредиты, Долги, Просрочки, Ипотека, Кредитная ...
text                     [(, Не переоценивайте свои финансовые возможно...
annotation_tags_chunk    [Кредиты, Долги, Просрочки, Ипотека, Кредитная...
text_chunk               [[Не переоценивайте свои финансовые возможност...
Name: 2, dtype: object

In [48]:
# Тестовый поиск по annotation
correct_id = 2 # рандомный идентификатор для примера annotation 
example = get_embedding(text=data.loc[correct_id, 'annotation']) # По тексту annotation получаем эмбединг
example = normalize_vector(example)

top_k_indices, top_k_similarities = hnsw_index_an_t.search(np.array([example], dtype=np.float32), 1) # В БД Tags+annotation ищем пример

assert correct_id == top_k_similarities.item(), 'Что то не работает =('

In [62]:
# Тестовый поиск по tags
correct_id = 2 # рандомный идентификатор для примера annotation 
example = get_embedding(text=data.loc[correct_id, 'tags']) # По тексту tags получаем эмбединг
example = normalize_vector(example)

top_k_indices, top_k_similarities = hnsw_index_an_t.search(np.array([example], dtype=np.float32), 10) # В БД Tags+annotation ищем пример

assert correct_id in top_k_similarities[0], 'Что то не работает =('